In [1]:
import torch
from torch import nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.nn import functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import time
import numpy as np
import json, os, math
from tqdm import tqdm

## 1. PATCH EMBEDDING


In [2]:
LEARNING_RATE = 0.003
BATCHSIZE = 512
PATCH_SIZE = 4
EMBED_DIM = 512
NUM_BLOCKS = 8
INTERMEDIATE_SIZE = 1 * EMBED_DIM
IMG_SIZE = 32
DROPOUT = 0.2
NUM_ATTENTION_HEADS = 8
NUM_CLASSES = 100
INIT_WEIGHT_RANGE = 0.02
NUM_EPOCHS = 100

In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.patch_size = PATCH_SIZE
        self.embed_dim = EMBED_DIM
        self.num_patch = int((IMG_SIZE * IMG_SIZE) / (PATCH_SIZE * PATCH_SIZE))
        self.projection = nn.Conv2d(in_channels=3, out_channels=EMBED_DIM, kernel_size=PATCH_SIZE, stride=PATCH_SIZE)
    def forward(self, x):
        B, C, H, W = x.shape
        x = self.projection(x).flatten(2).transpose(1, 2) # (B C H W), flatten thanh (B,C,H*W), transpose(1, 2) doi vi tri thanh B H*W C
        return x

## 2. POSITIONAL EMBEDDING

In [4]:
class PositionalEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.cls_token = nn.Parameter(torch.randn(1, 1, EMBED_DIM))
        self.patch_embed = PatchEmbedding()
        self.pos_embed = nn.Parameter(torch.randn(1, self.patch_embed.num_patch + 1, EMBED_DIM)) # N chieu + 1 chieu cho cls token
        self.dropout = nn.Dropout(DROPOUT)
        
    def forward(self, x):
        patch_embed = self.patch_embed(x) # B N D
        batch_size, _, _ = patch_embed.shape
        cls_token = self.cls_token.expand(batch_size, -1, -1) # nhân bản lên batchsize lần và giữ nguyên 2 chiều còn lại B, 1, D
        x = torch.cat((cls_token, patch_embed), dim = 1) # B, N + 1, D, trộn 2 cái vô theo chiều = 1
        x = x + self.pos_embed # pos + patch
        x = self.dropout(x)
        return x

## 3. ATTENTION HEAD

In [5]:
class AttentionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.num_attention_size = EMBED_DIM // NUM_ATTENTION_HEADS # 768 / 12 = 64
        self.query = nn.Linear(EMBED_DIM, self.num_attention_size) # 768 -> 64
        self.key = nn.Linear(EMBED_DIM, self.num_attention_size) # 768 -> 64
        self.value = nn.Linear(EMBED_DIM, self.num_attention_size) # 768 -> 64
        self.dropout = nn.Dropout(DROPOUT)
        
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        attention_w = torch.matmul(Q, K.transpose(-2, - 1)) / (self.num_attention_size ** 0.5)
        attention_w = torch.softmax(attention_w, dim=-1)
        attention_w = self.dropout(attention_w)
        output = torch.matmul(attention_w, V)
        return output, attention_w

## 4. MULTIHEAD ATTENTION

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed_dim = EMBED_DIM
        self.attention_head_size = EMBED_DIM // NUM_ATTENTION_HEADS # 768 / 12 = 64
        self.all_head_size = self.attention_head_size * NUM_ATTENTION_HEADS  # = 768
        self.heads = nn.ModuleList([])
        for _ in range(NUM_ATTENTION_HEADS):
            head = AttentionHead()
            self.heads.append(head)
        self.output_projection = nn.Linear(self.all_head_size, EMBED_DIM)
        self.output_dropout = nn.Dropout(DROPOUT)
    def forward(self, x):
        attention_outputs = [head(x) for head in self.heads]
        attention_output = torch.cat([attention_output for attention_output, _ in attention_outputs], dim=-1)
        attention_output = self.output_projection(attention_output)
        attention_output = self.output_dropout(attention_output)
        return attention_output

## 5. MLP

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(EMBED_DIM, INTERMEDIATE_SIZE)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(INTERMEDIATE_SIZE, EMBED_DIM)
        self.dropout = nn.Dropout(DROPOUT)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

## 6. BLOCK

In [8]:
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm1 = nn.LayerNorm(EMBED_DIM)
        self.mha = MultiHeadAttention()
        self.norm2 = nn.LayerNorm(EMBED_DIM)
        self.mlp = MLP()
        
    def forward(self, x):
        x_norm = self.norm1(x)
        attention_output = self.mha(x_norm)
        x = x + attention_output  # Residual connection
        # LayerNorm trước MLP
        x_norm = self.norm2(x)
        mlp_output = self.mlp(x_norm)
        x = x + mlp_output  # Residual connection
        return x

## 7. ENCODER

In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.ModuleList([])
        for _ in range(NUM_BLOCKS):
            block = Block()
            self.encoder.append(block)
    
    def forward(self, x):
        for block in self.encoder:
            x = block(x) # cho qua 12 block
        return x # B N + 1 D(embed dim)

## 8. ViT

In [10]:
class ViTClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = PositionalEmbedding()
        self.encoder = Encoder()
        self.norm = nn.LayerNorm(EMBED_DIM)  # Thêm LayerNorm trước classifier
        self.classifier = nn.Linear(EMBED_DIM, NUM_CLASSES)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.encoder(x)
        logits = self.classifier(x[:, 0, :]) # lay toan bo batch, phan tu 0 dau tien la cls token, toan bo dimension embed dim
        return logits

    def _init_weights(self, module):
        if isinstance(module, nn.Linear, nn.Conv2d):
            torch.nn.init.normal_(module.weight, mean=0.0, std=INIT_WEIGHT_RANGE)
        elif isinstance(module, nn.LayerNorm):
            module.weight.data.fill_(1.0)
        elif isinstance(module, PositionalEmbedding):
            module.position_embeddings.data = nn.init.trunc_normal_(
                module.position_embeddings.data.to(torch.float32),
                mean=0.0,
                std=INIT_WEIGHT_RANGE,
            ).to(module.position_embeddings.dtype)

            module.cls_token.data = nn.init.trunc_normal_(
                module.cls_token.data.to(torch.float32),
                mean=0.0,
                std=INIT_WEIGHT_RANGE,
            ).to(module.cls_token.dtype)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

train_set = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_set, batch_size=BATCHSIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=BATCHSIZE, shuffle=False, num_workers=2)
classes = tuple(train_set.classes)
print(classes)

100%|██████████| 169M/169M [00:02<00:00, 79.0MB/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
('apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 't

In [13]:
model = ViTClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=NUM_EPOCHS)

training_loss = []
training_acc = []
model.train()
# Training
for ep in tqdm(range(NUM_EPOCHS)):
    correct = 0
    total = 0
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        output = model(inputs)
        
        loss = criterion(output, labels)
        # backpropagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0) # cong voi so mau trong batch(batchsize)
        
    training_loss.append(running_loss/len(train_loader))
    training_acc.append((correct/total) * 100)
    scheduler.step()
    print(f"Epoch {ep + 1}: Training Loss: {running_loss/len(train_loader):.4f}, Training acc: {((correct/total) * 100):.2f}%")

  1%|          | 1/100 [01:55<3:11:17, 115.94s/it]

Epoch 1: Training Loss: 4.3994, Training acc: 4.10%


  2%|▏         | 2/100 [04:02<3:19:29, 122.14s/it]

Epoch 2: Training Loss: 4.0479, Training acc: 8.21%


  3%|▎         | 3/100 [06:08<3:20:39, 124.12s/it]

...

Epoch 98: Training Loss: 0.9614, Training acc: 71.09%


 99%|█████████▉| 99/100 [3:30:00<02:07, 127.45s/it]

Epoch 99: Training Loss: 0.9392, Training acc: 71.27%


100%|██████████| 100/100 [3:32:07<00:00, 127.28s/it]

Epoch 100: Training Loss: 0.9270, Training acc: 71.75%


In [14]:
torch.save(model.state_dict(), "bestweight.pth")

In [16]:
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
    
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test_loss: {test_loss/len(test_loader):.4f}, Test_acc: {(test_correct/test_total)*100 :.2f}")

Test_loss: 2.0845, Test_acc: 53.57
